# Example of the four-sphere head model

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from lfpykit.eegmegcalc import FourSphereVolumeConductor
from brainsignals.plotting_convention import mark_subplots, simplify_axes, cmap_v_e
from brainsignals.neural_simulations import return_equidistal_xyz

head_colors = ["#ffb380", "#74abff", "#b3b3b3", "#c87137"]

radii = [89000., 90000., 95000., 100000.]  # (µm)
sigmas = [0.276, 1.65, 0.01, 0.465]  # (S/m)

eps = 1e-2

dt = 0.1
tstop = 10
num_tsteps = int(tstop / dt + 1)
tvec = np.arange(num_tsteps) * dt

p_x = np.zeros(num_tsteps)
p_y = np.zeros(num_tsteps)
p_z = np.zeros(num_tsteps)

tau1 = 1
tau2 = 0.2
p_z[50:] = (np.exp(-tvec[:-50] / tau1) - np.exp(-tvec[:-50] / tau2))
p_z *= 1e7 / np.max(np.abs(p_z))

p = np.array([p_x, p_y, p_z]) # 1 timestep (nA µm)
dipole_loc = np.array([0., 0., 88800.])  # (µm)

num_points = 500
x, y, z = return_equidistal_xyz(num_points, radii[-1] - eps)
x = np.r_[x, 0]
y = np.r_[y, 0]
z = np.r_[z, radii[-1] - eps]

idxs = np.where(z > 0)[0]
x = x[idxs]
y = y[idxs]
z = z[idxs]

r_elecs = np.array([x, y, z]).T # (µm)

dist = np.sqrt(np.sum((dipole_loc[:, None] - 
                       r_elecs.T)**2, axis=0))
top_elec_idx = np.argmin(dist)

def plot_four_sphere_model(ax, radii, dipole_loc=None):
    
    radii_tweaked = [radii[0]] + [r for r in radii[1:]]
    for i in range(4):
        ax.add_patch(plt.Circle((0, 0), radius=radii_tweaked[-1 - i],
                                   color=head_colors[-1-i],
                                   fill=True, ec='k', lw=.1))
    #ax.add_patch(Ellipse((0, radii_tweaked[-1]), 2000, 500))
    if not dipole_loc is None:
        ax.arrow(dipole_loc[0], dipole_loc[2] - 1200, 0, 500, 
                 color='k', head_width=300)
        
    # mark 4-sphere head model layers
    ax.text(-14500, radii[0] - 6000, 'brain', ha="right", rotation=8)
    ax.text(-14500, radii[1] - 3500, 'CSF', ha="right", rotation=8)
    ax.text(-14500, radii[2] - 4000, 'skull', ha="right", rotation=8)
    ax.text(-14500, radii[3] - 5000, 'scalp', ha="right", rotation=8)

def plot_head_outline(ax, radius):
    circle_npts = 100
    head_x = radius * np.cos(np.linspace(0, 2 * np.pi, circle_npts))
    head_y = radius * np.sin(np.linspace(0, 2 * np.pi, circle_npts))
    patches = []
    right_ear = mpatches.FancyBboxPatch([radius + 5000, -15000], 
                                        3000, 30000,
        boxstyle=mpatches.BoxStyle("Round", pad=5000))
    patches.append(right_ear)

    left_ear = mpatches.FancyBboxPatch([-radius - 8000, -15000], 
                                       3000, 30000,
        boxstyle=mpatches.BoxStyle("Round", pad=5000))
    patches.append(left_ear)

    collection = PatchCollection(patches, facecolor='none', 
                                 edgecolor='k', alpha=1.0)
    ax.add_collection(collection)
    ax.plot(head_x, head_y, 'k', lw=1)
    ax.plot([radius], lw=1)

    ax.plot([-10000, 0, 10000], 
            [radius, radius + 10000, radius], 'k', lw=1)

    

In [ ]:
sphere_model = FourSphereVolumeConductor(r_elecs, radii, sigmas)
M = sphere_model.get_transformation_matrix(dipole_loc)  
eeg = 1000 * M @ p # (uV)
top_eeg = eeg[top_elec_idx]
max_time_idx = np.argmax(np.abs(top_eeg))

In [ ]:
fig = plt.figure(figsize=[6, 1.8])
fig.subplots_adjust(bottom=0.17, right=0.93, left=0.06, 
                    top=0.9, wspace=.31, hspace=0.65)
ax_geom = fig.add_subplot(131, frameon=False, aspect=1,
                         xticks=[], yticks=[],
                         xlim=[-14000, 14000],
                         ylim=[80000, 101000])

ax_p = fig.add_subplot(232, xlabel="time (ms)", ylabel="$P_z$ (nAm)")
ax_eeg1 = fig.add_subplot(235, xlabel="time (ms)", ylabel="EEG (µV)")

ax_eeg2 = fig.add_subplot(133, frameon=False, aspect=1,
                         xticks=[], yticks=[],
                         xlim=[-120000, 120000],
                         ylim=[-120000, 120000])


cax = fig.add_axes([0.96, 0.2, 0.012,  0.6])
plot_four_sphere_model(ax_geom, radii, dipole_loc)

ax_geom.plot(x[top_elec_idx], z[top_elec_idx], '_', lw=2, c='k', ms=10)

ax_p.plot(tvec, p_z / 1e6, c='k')
ax_eeg1.plot(tvec, top_eeg, c='k')
ax_p.axvline(tvec[max_time_idx], ls='--', c='gray')
ax_eeg1.axvline(tvec[max_time_idx], ls='--', c='gray')

vmax = np.floor(np.max(np.abs(eeg[:, max_time_idx])))
vmap = lambda v: cmap_v_e((v + vmax) / (2*vmax))
levels = np.linspace(-vmax, vmax, 40)

contourf_kwargs = dict(levels=levels, 
                       cmap=cmap_v_e, 
                       vmax=vmax, 
                       vmin=-vmax,
                       extend="both")

img = ax_eeg2.tricontourf(x, y, eeg[:, max_time_idx], **contourf_kwargs)
ax_eeg2.tricontour(x, y, eeg[:, max_time_idx], **contourf_kwargs)

order = np.argsort(z)

plot_head_outline(ax_eeg2, radii[-1])

# axis cross in panel A
ax_geom.annotate("", xy=(-11 * 1e3, 83 * 1e3),
             xycoords='data', xytext=(-6 * 1e3, 83 * 1e3), textcoords='data',
             arrowprops=dict(arrowstyle="<|-",
                             connectionstyle="arc3,rad=0", facecolor='black'),
             zorder=100)
ax_geom.annotate("", xy=(-11 * 1e3, 83 * 1e3),
             xycoords='data', xytext=(-11 * 1e3, 88 * 1e3), textcoords='data',
             arrowprops=dict(arrowstyle="<|-",
                             connectionstyle="arc3,rad=0", facecolor='black'),
             zorder=100)
ax_geom.plot(-11 * 1e3, 83 * 1e3, 'o', ms=6, mec='k', mfc=head_colors[0], zorder=101)
ax_geom.plot(-11 * 1e3, 83 * 1e3, 'kx', ms=4, zorder=101)
ax_geom.text(-8 * 1e3, 81 * 1e3, '$x$')
ax_geom.text(-13.5 * 1e3, 81.5 * 1e3, '$y$')
ax_geom.text(-13 * 1e3, 86* 1e3, '$z$')

# axis cross in panel D
ax_eeg2.annotate("", xy=(-110 * 1e3, -70 * 1e3),
             xycoords='data', xytext=(-70 * 1e3, -70 * 1e3), textcoords='data',
             arrowprops=dict(arrowstyle="<|-",
                             connectionstyle="arc3,rad=0", facecolor='black'),
             zorder=100)
ax_eeg2.annotate("", xy=(-110 * 1e3, -70 * 1e3),
             xycoords='data', xytext=(-110 * 1e3, -30 * 1e3), textcoords='data',
             arrowprops=dict(arrowstyle="<|-",
                             connectionstyle="arc3,rad=0", facecolor='black'),
             zorder=100)
ax_eeg2.plot(-110 * 1e3, -70 * 1e3, 'o', ms=6, mec='k', mfc='w', zorder=101)
ax_eeg2.plot(-110 * 1e3, -70 * 1e3, 'k.', ms=4, zorder=101)
ax_eeg2.text(-90 * 1e3, -89 * 1e3, '$x$')
ax_eeg2.text(-128 * 1e3, -46 * 1e3, '$y$')
ax_eeg2.text(-130 * 1e3, -85 * 1e3, '$z$')

cbar = plt.colorbar(img, cax=cax)
cbar.set_label("µV", labelpad=1)

cbar.set_ticks([-vmax, -vmax/2, 0, vmax/2, vmax])

simplify_axes(fig.axes)
mark_subplots([ax_geom, ax_eeg2], "AD", ypos=1.0, xpos=-0.1)
mark_subplots([ax_p, ax_eeg1], "BC", ypos=1.2, xpos=-0.1)

plt.savefig("four_sphere_example.pdf")